<a href="https://colab.research.google.com/github/sansin19/TfEstimatorHandsOn/blob/master/BstnPbWthEstmtor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow==2.0.0-rc0

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
import tensorflow as tf
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns_colors = sns.color_palette('colorblind')

tf.random.set_seed(1)

     |████████████████████████████████| 86.3MB 42.4MB/s 
     |████████████████████████████████| 4.3MB 25.3MB/s 
     |████████████████████████████████| 501kB 37.2MB/s 
  Found existing installation: tensorflow 1.14.0
    Uninstalling tensorflow-1.14.0:
      Successfully uninstalled tensorflow-1.14.0


In [0]:
boston_dataset = load_boston()

data_X = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
data_Y = pd.DataFrame(boston_dataset.target, columns=["target"])
data = pd.concat([data_X, data_Y], axis=1)

In [0]:
train, test = train_test_split(data, test_size=0.2, random_state=1)
print(len(train), "train examples")
print(len(test), "test examples")


404 train examples
102 test examples


In [0]:
def df_to_dataset(dataframe, shuffle_and_repeat=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle_and_repeat:
    ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.repeat()
    
  ds = ds.batch(batch_size)
  
  return ds

In [0]:
# define feature_columns as a list of all the features as numeric columns using functions from tf.feature_column
feature_columns = []
#data_X.keys()
for key in data_X.keys():
  feature_columns.append(tf.feature_column.numeric_column(key=key))

In [0]:
configs = tf.estimator.RunConfig(tf_random_seed=1)
model = tf.estimator.LinearRegressor( feature_columns = feature_columns,config =configs)
model.train(input_fn = lambda: df_to_dataset(train),steps=2500 )

INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp0ggu99_d', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2ff93cb710>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this wa

In [0]:
ev_res = model.evaluate(input_fn = lambda: df_to_dataset(test,shuffle_and_repeat=False))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-10-05T13:05:41Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp0ggu99_d/model.ckpt-2500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-10-05-13:05:42
INFO:tensorflow:Saving dict for global step 2500: average_loss = 40.825123, global_step = 2500, label/mean = 22.574509, l